## **Overview**
**we will explore how to develop a suite of different types of LSTM models for time series forecasting.**

* ### **Univariate LSTM Models**

1. Data Preparation
2. Vanilla LSTM
3. Stacked LSTM
4. Bidirectional LSTM
5. CNN LSTM
6. ConvLSTM


* ### **Multivariate LSTM Models**

* ### **Multi-Step LSTM Models**

* ### **Multivariate Multi-Step LSTM Models**



In [13]:
import matplotlib
import warnings
# 忽略警告
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score as r2

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
# from scikeras.wrappers import KerasRegressor # 回归神经网络
from numpy import array
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers import Dense
from keras.layers import Bidirectional
# univariate bidirectional lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from tensorflow.python.keras.layers import CuDNNLSTM
from keras.layers import Bidirectional
import keras.backend as K
import os
import glob
import tensorflow as tf
from tensorflow.python.keras.layers import Activation
from keras import optimizers
from pygame import mixer
import time
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

In [14]:
# def read_someone_day_file(day):
#     IMF = []
#     xlsx_file = '../../Data/CEEMDAN/SingleFactor/558606/'
#     xlsx_path = glob.glob(os.path.join(xlsx_file,'*.xlsx'))
#
#     # data_col_len = pd.read_excel('../../Data/CEEMDAN/SingleFactor/558606/CEEMDAN_58606 Station_g.xlsx',header=None)
#
#     for imf_len in range(len(data_col_len.columns)):
#         for file in xlsx_path:
#              print(file)
#              # print(os.path.basename(file))
#              imfs = pd.read_excel(file,header=None)
#              imfn = imfs.iloc[:,imf_len]
#              imfn.to_numpy()
#              IMF.append(imfn )
#         print(imf_len,"-------------------")
#     return IMF

In [15]:
def Result_Ana(Model, feature_num,testX,testY,scaler):
    print('Result_Ana_Function:',testX.shape)
    y_pred = Model.predict(testX)
    y_pred = np.array(y_pred).reshape(-1, 1)
    # 将一个数组prediction在最后一个轴上（即axis=-1）进行复制，重复8次，并将结果存储在prediction_copies_array数组中
    # prediction_copies_array = np.repeat(y_pred, feature_num,axis=-1)
    # prediction_copies_array
    # print('-----------prediction copy Shape：',prediction_copies_array.shape,'------------------')
    # Multi Var prediction
    pred = scaler.inverse_transform(y_pred) #进行逆变换但是，只需要最后一列
    # Single prediction
    # pred = scaler.inverse_transform(y_pred) #进行逆变换但是，只需要最后一列

    # print(pred)
    print('testY of shape :',testY.shape)
    y_true = np.array(testY).reshape(-1, 1)
    # original_copies_array = np.repeat(testY, feature_num, axis=-1)
    # print('IMF_True:',scaler.inverse_transform(np.reshape(original_copies_array, (len(testY), feature_num))))
    y_true = scaler.inverse_transform(y_true)

    # print('feature_num-1_ETO_true:',y_true)

    # Single true value
    # y_true = scaler.inverse_transform(np.array(testY).reshape(-1,1))

    # y_true
    # pred

    # print(pred)
    plt.plot(y_true, color='red', label='Real Value')
    plt.plot(pred, color='blue', label='Pred Value')
    plt.title('Prediction ETO (mm)')
    plt.xlabel('Time (day)')
    plt.ylabel('ETO (mm)')
    plt.legend()
    plt.show()


    print('MSE:',mse(y_true,pred))
    print('MAE:',mae(y_true,pred))
    print('R²:',r2(y_true,pred))
    print('RMSE:',np.sqrt(mse(y_true,pred)))
    # print('pred_finally:',pred)

    return y_true,pred

In [16]:
IMF = pd.read_excel('../../Data/CEEMDAN/SingleFactor/558606/'+'CEEMDAN_58606 Station_'+'nine'+'.xlsx',header=None)

In [17]:
IMF

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.079223,0.106928,-0.080420,-0.556599,-0.048314,-0.451930,-0.209208,-0.298211,-0.178119,-0.177663,0.123194,0.129041,2.966678
1,0.167664,0.061526,-0.061378,-0.577853,-0.040365,-0.462905,-0.213679,-0.299652,-0.177326,-0.176424,0.123444,0.129086,2.966663
2,-0.110238,-0.007235,-0.102145,-0.605239,-0.012286,-0.470823,-0.216857,-0.300535,-0.176437,-0.175176,0.123693,0.129131,2.966647
3,-0.033192,-0.042988,-0.237289,-0.597712,0.036943,-0.475983,-0.218702,-0.300854,-0.175452,-0.173919,0.123941,0.129175,2.966632
4,-0.071789,-0.013570,-0.356966,-0.503781,0.103794,-0.478906,-0.219169,-0.300612,-0.174372,-0.172653,0.124189,0.129219,2.966616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,0.554206,0.035548,0.309859,0.511260,0.509747,0.480694,0.091120,0.040725,0.000500,-0.142435,0.164101,0.107699,2.888278
2721,0.686508,0.095871,0.131412,0.511031,0.568968,0.489362,0.094142,0.043953,0.000890,-0.142090,0.163991,0.107703,2.888260
2722,0.645852,-0.021438,-0.146584,0.475814,0.585492,0.494823,0.096659,0.047117,0.001277,-0.141743,0.163879,0.107708,2.888242
2723,-1.017044,-0.166530,-0.393899,0.425741,0.563115,0.496723,0.098610,0.050218,0.001660,-0.141396,0.163766,0.107712,2.888224


In [18]:
IMF[0]

0       0.079223
1       0.167664
2      -0.110238
3      -0.033192
4      -0.071789
          ...   
2720    0.554206
2721    0.686508
2722    0.645852
2723   -1.017044
2724   -0.573783
Name: 0, Length: 2725, dtype: float64

In [19]:
len(IMF.columns)

13

In [20]:
np.array(IMF)

array([[ 0.07922266,  0.10692759, -0.0804196 , ...,  0.1231943 ,
         0.1290414 ,  2.9666781 ],
       [ 0.16766371,  0.06152574, -0.06137822, ...,  0.12344377,
         0.12908604,  2.96666263],
       [-0.11023805, -0.00723506, -0.10214502, ...,  0.12369274,
         0.12913056,  2.96664714],
       ...,
       [ 0.64585228, -0.02143841, -0.14658354, ...,  0.16387918,
         0.10770784,  2.88824221],
       [-1.01704402, -0.16652988, -0.39389851, ...,  0.16376619,
         0.10771212,  2.8882243 ],
       [-0.57378275, -0.09043074, -0.28334668, ...,  0.16365182,
         0.1077162 ,  2.88820643]])

In [21]:
np.transpose(IMF)

,0,1,2,3,4,5,6,7,8,9,...,2715,2716,2717,2718,2719,2720,2721,2722,2723,2724
0,0.079223,0.167664,-0.110238,-0.033192,-0.071789,0.108033,0.134707,0.078454,-0.446654,0.487440,...,-0.351161,0.846940,0.625956,-0.859787,0.039442,0.554206,0.686508,0.645852,-1.017044,-0.573783
1,0.106928,0.061526,-0.007235,-0.042988,-0.013570,0.037879,0.035816,-0.033117,-0.028879,0.039557,...,0.014994,0.022176,0.007857,-0.009390,-0.033962,0.035548,0.095871,-0.021438,-0.166530,-0.090431
2,-0.080420,-0.061378,-0.102145,-0.237289,-0.356966,-0.417812,-0.429407,-0.283760,-0.044046,0.358733,...,-0.311115,-0.288907,-0.097090,0.073601,0.301870,0.309859,0.131412,-0.146584,-0.393899,-0.283347
3,-0.556599,-0.577853,-0.605239,-0.597712,-0.503781,-0.349364,-0.150800,0.131835,0.425981,0.619182,...,0.177511,0.264896,0.384458,0.451318,0.483405,0.511260,0.511031,0.475814,0.425741,0.386711
4,-0.048314,-0.040365,-0.012286,0.036943,0.103794,0.178201,0.246775,0.293613,0.308375,0.286457,...,-0.106149,0.025202,0.159267,0.290102,0.411381,0.509747,0.568968,0.585492,0.563115,0.507279
5,-0.451930,-0.462905,-0.470823,-0.475983,-0.478906,-0.480113,-0.479859,-0.478459,-0.475966,-0.471793,...,0.390725,0.414432,0.435634,0.453814,0.468858,0.480694,0.489362,0.494823,0.496723,0.495011
6,-0.209208,-0.213679,-0.216857,-0.218702,-0.219169,-0.218225,-0.215877,-0.212166,-0.207158,-0.200939,...,0.069712,0.074761,0.079445,0.083747,0.087645,0.091120,0.094142,0.096659,0.098610,0.099941
7,-0.298211,-0.299652,-0.300535,-0.300854,-0.300612,-0.299823,-0.298503,-0.296668,-0.294338,-0.291528,...,0.023755,0.027243,0.030690,0.034090,0.037436,0.040725,0.043953,0.047117,0.050218,0.053253
8,-0.178119,-0.177326,-0.176437,-0.175452,-0.174372,-0.173199,-0.171934,-0.170581,-0.169140,-0.167614,...,-0.001497,-0.001093,-0.000691,-0.000291,0.000106,0.000500,0.000890,0.001277,0.001660,0.002038
9,-0.177663,-0.176424,-0.175176,-0.173919,-0.172653,-0.171378,-0.170094,-0.168802,-0.167501,-0.166192,...,-0.144147,-0.143806,-0.143465,-0.143123,-0.142779,-0.142435,-0.142090,-0.141743,-0.141396,-0.141048


In [22]:
def ceemdan_lstm(day):
    mixer.init()
    mixer.music.load('../../Resources/ExecuteTipAudio.mp3')

    reshape1 = 0
    IMF = pd.read_excel('../../Data/CEEMDAN/SingleFactor/558606/'+'CEEMDAN_58606 Station_'+day+'.xlsx',header=None)

    imf_choose = 0
    finally_rsult = []
    finally_y_true = []

    for imf_run in range(int(len(IMF.columns))):
        # print(imf_run)
        # print(IMF[0])
        Single_Factor_IMF = IMF[imf_run]

        print('--------------------------',imf_run,'--------------------------')
        Single_Factor_IMF = np.array(Single_Factor_IMF)
        # IMF_Input = np.transpose(Single_Factor_IMF)

        df_IMF = pd.DataFrame(Single_Factor_IMF)
        df_IMF.columns = ['ET0']

        # print(df_IMF)

        scaler = MinMaxScaler(feature_range=(0, 1))
        df = scaler.fit_transform(df_IMF)

        sacler_data_len = len(df_IMF.columns)

        #数据集划分
        test_split = round(len(df) * 0.20)
        print(test_split)
        def splitData(var, per_test):
            num_test = int(len(var) * per_test)
            train_size = int(len(var) - num_test)
            train_data = var[0:train_size]
            test_data = var[train_size:train_size + num_test]
            return train_data, test_data


        df_training, df_testing = splitData(df, 0.2)
        print('df_training.shape:',df_training.shape)
        print('df_testing.shape:',df_testing.shape)

        def createXY(data,n_past,n_steps_out):
            dataX,dataY = list(),list()

            for i in range(len(data)):
                ## such as: len(data) = 19624
                end_ix = i + n_past ## 0+3=3,1+3=4,...,19618+3=19621
                out_end_ix = end_ix + n_steps_out ## 3+3=6,4+3=7,...,19621+3=19624
                # print('out_end_ix:',out_end_ix)
                '''
                加入此处判断，使得最终的dataX和dataY中每行的数组长度一致，进而可以转换为array(数组),如若不加以限制，
                则导致最后几次循环由于i是在len(data)范围内的，
                但是由于out_end_ix=end_ix + n_steps_out,最终会超出data的数据范围，而导致其无法获取到数据,
                加入dataY之后，最后几行的数据长度是和前面的数据长度不一致，最终导致无法进行array转换
                '''
                if out_end_ix > len(data): ## 6 < len(data),7<len(data),...,19623+3=19626>len(data)=19624
                    # print("------------out_end_ix of end",out_end_ix,'---------------')
                    break

                dataX.append(data[i:end_ix,0]) ## 0:3,0:7;1:4,0:7
                # print('dataX:---------------')
                # print(data[i:end_ix,0])
                # print('dataX:---------------')
                dataY.append(data[end_ix:out_end_ix,data.shape[1]-1]) ##3:6,6;4:7,6
                # print('dataY:----------------------')
                # print(data[end_ix:out_end_ix,data.shape[1]-1])
                # print('dataY:----------------------')
            return np.array(dataX), np.array(dataY)

        n_past=1
        n_steps_out=1

        trainX, trainY = createXY(df_training, n_past, n_steps_out)
        testX, testY = createXY(df_testing, n_past, n_steps_out)

        reshape1 = testY.shape[0]

        print('trainX.Shape:----', trainX.shape)
        print('trainY.shape:----', trainY.shape)
        print('testX.shape:----', testX.shape)
        print('testY.shape:----', testY.shape)

        # trainY = trainY.reshape(-1)
        # testY = testY.reshape(-1)
        # print('trainY Shape 2 ---', trainY.shape)
        # print('testY Shape 2 ---', testY.shape)
        # trainX = np.expand_dims(trainX,axis=1)
        # testX = np.expand_dims(testX,axis=1)
        # print('expand_dims_trainX of shape',trainX.shape)
        # print('expand_dims_testX of shape',testX.shape)

        print('----------------------------',imf_run,'------------------------------')

        def N_LSTM(batch_size=32, epochs=40):
                model = Sequential()
                model.add(LSTM(200, activation='relu', input_shape=(n_past, 1)))
                # model.add(LSTM(100,activation='relu'))
                #model.add(CuDNNLSTM(200, return_sequences=True, input_shape=(None, sacler_data_len)))

                # model.add(LSTM(50, activation='relu'))
                model.add(Dropout(0.1))
                # model.add(LSTM(50, activation='relu'))
                # model.add(Dropout(0.1))

                # opm_adam = Adam(lr=0.01)  # 设置为您希望的学习率
                # model.add(Dense(1))
                ## Full connection layer: This is output shape(e.g. 3 Dimension that trainY.shape[1] = 3 represent for each output is 3 number)
                model.add(Dense(1))

                model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

                return model

        N_LSTM_Model = KerasRegressor(N_LSTM, epochs=40, verbose=1, validation_data=(testX, testY))
        # N_LSTM_Model = KerasRegressor(N_LSTM, epochs=40, verbose=1)

        from sklearn.model_selection import GridSearchCV

        # 定义超参数搜索范围
        param_grid = {
            # 'optimizer': ['adam'],
            'batch_size': [128, 256],
            'epochs': [20,30]
            # 'batch_size': [64],
            # 'epochs': [40]
        }

        # 执行网格搜索
        grid = GridSearchCV(estimator=N_LSTM_Model, param_grid=param_grid, cv=2)
        grid_result = grid.fit(trainX, trainY)


        best_params = grid_result.best_params_

        best_params
        print('prediction' , 'one' ,'day best_params:', best_params)

        best_model = grid_result.best_estimator_

        y_trues,prediction_result = Result_Ana(best_model, sacler_data_len,testX,testY,scaler)

        print('prediction_result:',prediction_result)
        finally_rsult.append(prediction_result)
        print('lstm_y_turs:',y_trues)
        finally_y_true.append(y_trues)

        mixer.music.play()
        time.sleep(1)
        mixer.music.stop()
    return reshape1,finally_y_true,finally_rsult,day

In [23]:
def plot_save_true_prediction(reshape1,finally_y_true,finally_rsult,day):
    # WL(Water Level(m)) For IMF1-IMF13-R True Value
    finally_y_true
    finally_ytrue_r = np.array(finally_y_true)
    finally_ytrue_r
    reshape2 = len(finally_ytrue_r)
    reshape2
    finally_column_names = []

    for i in range(1, reshape2 + 1):
        if i < (reshape2):
            finally_column_names.append("IMF" + str(i))
        else:
            finally_column_names.append("Residual")
    finally_column_names
    ALL_Factor_ReIMFS_Yture_Result = np.transpose(finally_ytrue_r)
    reshape1
    ALL_Factor_ReIMFS_Yture_Result = ALL_Factor_ReIMFS_Yture_Result.reshape(reshape1, reshape2)
    all_factor_ceemdan_true = pd.DataFrame(ALL_Factor_ReIMFS_Yture_Result, columns=finally_column_names)
    all_factor_ceemdan_true
    sums = all_factor_ceemdan_true.iloc[:, :].sum(axis=1)

    # 将求和结果添加到DataFrame中作为新的一列
    all_factor_ceemdan_true["True"] = sums
    # 保存为CSV文件
    true_file_path = "Result/CEEMDAN_Single_Factor/"+day+"_day_CEEMDAN_IMF" + str(reshape2 - 1) + "-LSTM_true.csv"

    all_factor_ceemdan_true.to_csv(true_file_path,
                                   index=False)
    # WL(Water Level(m)) IMF1-IMF13-R Prediction Value
    finally_rsult
    finally_r = np.array(finally_rsult)
    finally_r
    ALL_Factor_ReIMFS_Result = np.transpose(finally_r)
    ALL_Factor_ReIMFS_Result = ALL_Factor_ReIMFS_Result.reshape(reshape1, reshape2)
    finally_column_names
    all_factor_ceemdan_prediction = pd.DataFrame(ALL_Factor_ReIMFS_Result, columns=finally_column_names)
    all_factor_ceemdan_prediction
    sums = all_factor_ceemdan_prediction.iloc[:, :].sum(axis=1)

    # 将求和结果添加到DataFrame中作为新的一列
    all_factor_ceemdan_prediction["Pred"] = sums
    prediction_file_path = "Result/CEEMDAN_Single_Factor/"+day+"_day_CEEMDAN_IMF" + str(reshape2 - 1) + "-LSTM_prediction.csv"
    # 保存为CSV文件
    all_factor_ceemdan_prediction.to_csv(prediction_file_path, index=False)
    ### 验证最终结果
    df = pd.read_csv(prediction_file_path)
    df = df[['Pred']]
    df

    origin = pd.read_csv(true_file_path)
    origin = origin[['True']]
    # y_true = origin.iloc[:,len(origin.columns)-1]
    y_true = origin
    y_true
    pred = df
    y_true
    pred
    plt.plot(y_true, color='red', label='Real Value')
    plt.plot(pred, color='blue', label='Pred Value')
    plt.title('Prediction ET0 ')
    plt.xlabel('Time')
    plt.ylabel('Detail Value')
    plt.legend()

    plt.show()

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    from sklearn.metrics import r2_score as r2

    print('MSE:', mse(y_true, pred))
    print('MAE:', mae(y_true, pred))
    print('R²:', r2(y_true, pred))
    print('RMSE:', np.sqrt(mse(y_true, pred)))
    print('pred_finally:', pred)

In [ ]:
%%time
if __name__ == '__main__':
    days = ['one','three','five','seven','nine','ten','fifteen']
    for day in days:
       print('------------------',day ,'--------------------')
       print('This is ',day,'-th for prediction et0')
       reshape1,finally_y_true,finally_rsult,day = ceemdan_lstm(day)
       print('This is ',day,'-th for prediction et0')
       print('------------------',day,'---------------------')
       plot_save_true_prediction(reshape1,finally_y_true,finally_rsult,day)